In [11]:
import pandas as pd
import numpy as np
import requests
import json
import datetime
from IPython.display import display
from sklearn import cluster, linear_model
from scipy.optimize import curve_fit
import plotly.graph_objs as go
import plotly.plotly as py
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode() 

def get_dates_for_fiscal_year(fiscal_year):
    year = int('20{}'.format(fiscal_year[:2]))
    summer_start = datetime.date(year, 4, 1)
    summer_end = datetime.date(year, 9, 30)
    winter_start = datetime.date(year, 10, 1)
    winter_end = datetime.date(year+1, 3, 31)
    return winter_start, winter_end, summer_start, summer_end

def get_fiscal_year(row):
    if row[-1].month > 3: return str(row[-1].year)[-2:]+str(row[-1].year+1)[-2:]
    else: return str(row[-1].year-1)[-2:]+str(row[-1].year)[-2:]

In [13]:
# load a map of states abbreviations from github
states_response = requests.get(
    'https://gist.githubusercontent.com/mshafrir/2646763/raw/8b0dbb93521f5d6889502305335104218454c2bf/states_hash.json')
json_states = json.loads(states_response.text)

eia_website_up = True

if eia_website_up: # login via HTTP API
    # Get data FROM U.S. Energy Information Administration using the below API key
    api_key = '508B0FE56B9D2507A2CE82A700EE86D0'
    req = 'http://api.eia.gov/series/?api_key={}&series_id=NG.N3010{}2.M'.format(api_key,'US')
    r_us = requests.get(req)
    data_us = json.loads(r_us.text)
    dates = []
    volumes = []
    # set index using the total U.S. data
    for element in data_us['series'][0]['data']:
        dates.append(datetime.datetime.strptime(element[0], '%Y%m').date())
        volumes.append(float(element[1]))    
    df = pd.DataFrame(index=dates)
    df.sort_index(ascending=True, inplace=True)

    # load states consumption volumes into dataframe
    for state in json_states:
        dates = []
        volumes = []
        r = requests.get('http://api.eia.gov/series/?api_key={0}&series_id=NG.N3010{1}2.M'.format(api_key, state))
        try:
            data = json.loads(r.text)
            column_name = json_states[data['series'][0]['geography'].split('-')[1]]
            for element in data['series'][0]['data']:
                dates.append(datetime.datetime.strptime(element[0], '%Y%m').date())
                volumes.append(float(element[1]))                
            df_tmp = pd.DataFrame(index=dates)
            df_tmp[column_name] = volumes
            df = pd.merge(df, df_tmp, how='outer', left_index=True, right_index=True)
        except:
            continue
else: # RELY ON CSV FILE
    # EIA.GOV under maintenance...
    df = pd.read_csv('./NG_CONS_SUM_A_EPG0_VGT_MMCF_M.csv', delimiter=';', skiprows=2)
    df.drop(df.columns[1],1,inplace=True) # drop US column
    new_columns = []
    for state in json_states:
        new_columns.append(json_states[state])
    new_columns = sorted(new_columns)
    new_columns.remove('American Samoa') 
    new_columns.remove('Federated States Of Micronesia')
    new_columns.remove('Guam')
    new_columns.remove('Marshall Islands')
    new_columns.remove('Northern Mariana Islands')
    new_columns.remove('Palau')
    new_columns.remove('Puerto Rico')
    new_columns.remove('Virgin Islands')
    df.columns = ['Date']+new_columns
    df.set_index('Date', inplace=True)

df = df.dropna(how='all')
# build a normalized dataframe
df_norm = df.apply(lambda x: x/x.sum())

# total data
total_data = [go.Scatter(x=df.index, y=df[c], name=c, mode='lines') for c in df.columns]
total_layout = dict(title='Domestic US Gas consumption',
                   xaxis=dict(title='Delivery Month'),
                   yaxis=dict(title='Consumption (MMcft)'))
fig_tot=dict(data=total_data, layout=total_layout)

# Add fiscal year column
df['DeliveryDate'] = pd.to_datetime(df.index)
df.insert(0,'FiscalYear',df.apply(get_fiscal_year, axis=1))
df=df.drop('DeliveryDate',1)
df_norm['DeliveryDate'] = pd.to_datetime(df_norm.index)
df_norm.insert(0,'FiscalYear',df_norm.apply(get_fiscal_year, axis=1))
df_norm=df_norm.drop('DeliveryDate',1)

In [5]:
first_year = df_norm.first_valid_index().year
last_year = df_norm.last_valid_index().year
fiscal_years = ['{0}{1}'.format(str(year)[-2:],str(year+1)[-2:]) for year in range(first_year, last_year)]
tuples = []
for state in df_norm.columns[1:]:
    for fy in fiscal_years:
        tuples.append((state,fy))
ix = pd.MultiIndex.from_tuples(tuples)
df_bystate = pd.DataFrame(index=ix, columns=['Seasonality'])
df_bystate.index.names = ['State', 'FiscalYear']

for state in df_bystate.index.get_level_values('State'):
    for fy in df_bystate.index.get_level_values('FiscalYear'):
        max_value = df_norm[state][df['FiscalYear']==fy].max()
        min_value = df_norm[state][df['FiscalYear']==fy].min()
        df_bystate.ix[state, fy]['Seasonality'] = (max_value-min_value)/2.0

df_bystate['Seasonality'] = df_bystate['Seasonality'].astype(float)
df_bystate=df_bystate.groupby(level=0).describe()
df_bystate.index.names = ['State', 'Value']
df_bystate = df_bystate.xs('mean', level='Value')

df_var = df[df.columns[1:]].pct_change()
stds = []
for state in df_bystate.index:
    stds.append(df_var[state].std())
df_bystate['MonthStd'] = stds

KeyboardInterrupt: 

In [83]:
# LINEAR REGRESSION
x_values = df_bystate['MonthStd'].values
y_values = df_bystate['Seasonality'].values
regr = linear_model.LinearRegression()
regr.fit(x_values.reshape(len(x_values),1), y_values.reshape(len(y_values),1))
regr2 = linear_model.Ridge(alpha=0.5)
regr2.fit(x_values.reshape(len(x_values),1), y_values.reshape(len(y_values),1))
y_predict = regr.predict(x_values.reshape(len(x_values),1))
y_predict2 = regr2.predict(x_values.reshape(len(x_values),1))
predicted_data = [go.Scatter(x=x_values, y=y_predict[:,0], mode='lines', name = 'linear model')]
actual_data = [go.Scatter(x=df_bystate['MonthStd'], y=df_bystate['Seasonality'], mode='markers',
                         name = 'State data')]
data = actual_data+predicted_data
layout = dict(title='U.S.A. natural gas residential consumption by state',
             xaxis = dict(title='Monthly variations standard deviation'),
             yaxis = dict(title='Average winter-summer consumption spread'),
             annotations=[dict(x=0.3,y=0.0005,xref='x',yref='y',
                               text='source: U.S. Energy Information Administration (https://www.eia.gov)',
                               font=dict(size=10),showarrow=False)])
fig_reg=dict(data=data, layout=layout)

In [80]:
df_bystate['Indicator'] = df_bystate['Seasonality']*df_bystate['MonthStd']
riskiest_3 = df_bystate.sort_values(by='Indicator', ascending=False)[:3]
safest_3 = df_bystate.sort_values(by='Indicator', ascending=False)[-3:]
riskiest_3_data = [go.Scatter(x=df_norm.index, y=df_norm[c], name=c, mode='lines', 
                              marker=dict(color='darkred'), showlegend=False) for c in riskiest_3.index.values]
safest_3_data = [go.Scatter(x=df_norm.index, y=df_norm[c], name=c, mode='lines', marker=dict(color='darkgreen'),
                           showlegend=False) for c in safest_3.index.values]
fake_1 = [go.Scatter(x=[1],y=[1],name='Risky',marker=dict(color='darkred'),mode='lines',visible='legendonly')]
fake_2 = [go.Scatter(x=[1],y=[1],name='Safe',mode='lines',marker=dict(color='darkgreen'),visible='legendonly')]
layout = dict(title="Three 'riskiest' and 'safest' natural gas consumption profiles",
              xaxis=dict(title='Delivery Month'),
              yaxis=dict(title='Normalized volume')
            )
fig_risk=dict(data=riskiest_3_data+safest_3_data+fake_1+fake_2, layout=layout)

In [14]:
py.sign_in('enrico.tesio','ttv7w7heht')
#url_reg=py.iplot(fig_reg, filename='gas-consumption-regression', sharing='public')
#url_risk=py.iplot(fig_reg, filename='gas-consumption-regression', sharing='public')
url_consumption = py.iplot(fig_tot, filename='total-domestic-consumption', sharing='public')